In [ ]:
# CSCI 381/780 (Fall 2022) - Project 1

**Due Date: Friday, October 7 by 4 PM**

## Description
In this project you will construct machine learning models on a dataset comprised of sociodemographic data of U.S. citizens collected from the 1994 U.S. Census. The machine learning task will be binary classification of people based on their income level being either <= 50,000 USD or > 50,000 USD.

## Instructions
1. In this project you will write code to construct machine learning models using various partitions (see the figure below) of the census dataset and write responses to questions concerning the performance of said models. Please complete all sections below, adding new *Code* or *Markdown* cells as appropriate to answer the questions.
2. There are many Scikit-learn functions that leverage randomness to generate results. For these functions, a pseudorandom generator can be intialized using a seed value by passing the parameter `random_state=XXX`, where `XXX` is some number between 1 and 2^31-1. For each of these functions, **you will utilize your CUNY ID number** to initialize the function. Functions include:
- `StratifiedShuffleSplit`
- `RandomForestClassifier`
- `RandomizedSearchCV`
3. You will **work independently** on the project. Please make use of the *Python Data Science Reference Materials* posted on Blackboard, or come to office hours should you need further assistance.
4. You will submit a single Jupyter notebook containing all code and written responses via Blackboard by the due date listed above. 

<img src="project-1-data-folds.png" width="600" height="300">

## Grading

### Running Code
Your Jupyter notebook must be able to run from start to finish **without error**. Please turn any cell that contains scratch work or other non-executable items to *Raw*. **Notebooks that cannot run to completion will receive a grade of 0**.

### Holdout Set Evaluation
Your final models will be evaluated against a holdout set. You model performances with respect to AUC on this set must be *comparable* (e.g., within 5%) of those reported in Part 6.

### Rubric

|**Part**|0|1|2|3|4|5|6|7|**Total**|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**%**|15|5|10|20|10|10|20|10|100|


## Part 0: Load Data

In [ ]:
# Set the variable `CENSUS_FILE` to the **full path** to the census dataset (**census_dataset.csv**) on your system. Load the file into a dataframe (you may intialize the column names using the header list `census_column_names`), then:
1. Determine the number and types of features.
2. Determine the number of classes and their prevalence in the dataset. Are the classes balanced?
3. Perform any necessary preprocessing on dataset.
4. Perform a **stratified split** of the data into training/validation/test sets, 60%/20%/20%. 
5. Verify that the training/validation/test splits have the same prevalence as the original dataset.
6. Standardize the training/validation/test splits (fit on the training, then transform the validation/test sets). Use the standardized splits for the SVM models *only*.

In [ ]:
census_column_names=['age','workclass','fnlwgt','education','education-num',
                      'marital-status','occupation','relationship','race','sex',
                      'capital-gain','capital-loss','hours-per-week','native-country',
                      'annual-income']

## Part 1: Train Initial Models

In [ ]:
Using default hyperparameters:
1. Construct **Naive Bayes (NB)**, **Support Vector Machine (SVM)** , and **Random Forest (RF)** models on the training set.
2. Calculate the confusion matrix and report the following performance metrics on the **training set**:
    *Accuracy*, *F1 Score*, *AUC*, *Sensitivity*, *Specificity*, and *Precision*. You can use the function `p1_metrics` for this purpose. Are any of the models underfitting the data? Is so, why?

3. Calculate the same metrics by applying the trained model to the **validation set**. Compare and contrast the errors each model makes in terms of each class. 

In [ ]:
#peformance metric functions
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
import numpy as np

#A list of keys for the dictionary returned by p1_metrics
metric_keys = ['auc','f1','accuracy','sensitivity','specificity', 'precision']

def p1_metrics(y_true,y_pred,include_cm=True):
    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    if include_cm:
        return {
            'auc': roc_auc_score(y_true,y_pred),
            'f1': f1_score(y_true,y_pred),
            'accuracy': (tp+tn)/np.sum(cm),
            'sensitivity': tp/(tp+fn),
            'specificity': tn/(tn+fp),
            'precision': tp/(tp+fp),
            'confusion_matrix': cm}
    else:
        return {
            'auc': roc_auc_score(y_true,y_pred),
            'f1': f1_score(y_true,y_pred),
            'accuracy': (tp+tn)/np.sum(cm),
            'sensitivity': tp/(tp+fn),
            'specificity': tn/(tn+fp),
            'precision': tp/(tp+fp)}

#This wrapper can be used to return multiple performance metrics during cross-validation
def p1_metrics_scorer(clf,X,y_true):
    y_pred=clf.predict(X)
    return p1_metrics(y_true,y_pred,include_cm=False)

## Part 2: Cross-Validation Models

In [ ]:
Split the **non-test data** (*training* + *validation* data) into **stratified 5-folds** for cross-validation purposes, then:
1. Train NB, SVM, and RF models using 5-fold cross-validation.
2. Report the mean and standard deviation of the performance metrics listed in Part 1.2 for each model. You may use the function `collate_cv_results` for this purpose.
3. How does the performance of these models compare with those created in Part 1? Which models' performances are more consistent, and why?

In [ ]:
#Summarizes model performance results produced during cross-validation
def collate_cv_results(cv_results,display=True):
    cv_stats=dict()
    for k in cv_results:
        cv_stats[k+"_mean"]=np.mean(cv_results[k])
        cv_stats[k+"_std"]=np.std(cv_results[k])
        if display:
            print(k,cv_stats[k+"_mean"],"("+str(cv_stats[k+"_std"])+")")
    return cv_stats

## Part 3: SVM Hyperparameter Tuning

In [ ]:
Utilizing the cross-validation dataset from Part 2, you will construct SVM models using the below strategies:
1. Using `GridSearchCV`, determine the best choice of hyperparameters out of the following possible values:
- *Kernel type*: Linear, radial basis function
- *Box constraint (C)*: [1, 5, 10, 20]
- *Kernel width (gamma)*: 'auto','scale'
2. Report the time required to perform cross-validation via `GridSearchCV`. Report the mean and standard deviation of the performance metrics for the best performing model along with its associated hyperparameters. You may use the function `collate_ht_results` for this purpose.
3. Using `RandomizedSearchCV`, determine the best choice of hyperparameters after **16 trials** using the same possible values for *kernel type* and *kernel width* but with possible **box constraint** values of [1,5,10,15,....,100]. 
4. Report the time required to perform cross-validation via `RandomizedSearchCV`. Report the mean and standard deviation of the performance metrics for the best performing model along with its associated hyperparameters.
5. Compare the performance of the two models (using the metrics in 1.2) as well as the time required to compute the models. Which hyperparameter method do you think is better in this case, and why?

### Please Read!
There are a few parameters for the `GridSearchCV` and `RandomizedSearchCV` functions that should be set:
- `scoring` - This controls the strategy to evaluate the performance of the cross-validated model on the test set, set it to `p1_metrics_scorer`.
- `refit` - This will refit an estimator using the best found parameters on the whole dataset, set it to `"auc"`
- `cv` - This will enable you to reuse your CV splits from Part 2.
    `n_jobs` - Number of jobs to run in parallel, if you have more than one core on your device (you should), set this to as many as you'd like to use, or to `-1` if you want to use all available cores.
- `return_train_score` - Setting this to `False` will reduce computational time.

In [ ]:
#Summarizes model performance results produced during hyperparameter tuning
def collate_ht_results(ht_results,metric_keys=metric_keys,display=True):
    ht_stats=dict()
    for metric in metric_keys:
        ht_stats[metric+"_mean"] = ht_results.cv_results_["mean_test_"+metric][ht_results.best_index_]
        ht_stats[metric+"_std"] = metric_std = ht_results.cv_results_["std_test_"+metric][ht_results.best_index_]
        if display:
            print("test_"+metric,ht_stats[metric+"_mean"],"("+str(ht_stats[metric+"_std"])+")")
    return ht_stats

## Part 4: Random Forest Hyperparameter Tuning

Utilizing the cross-validation dataset from Part 2, construct a Random Forest model:

1. Using `RandomizedSearchCV`, determine the best choice of hyperparameters after **16 trials** using the following possible values:
- *Split criterion*: Gini impurity, information gain (entropy)
- *Maximum tree depth*:None, log<sub>2</sub>|cross-validation dataset|-1 (use `ceil` to round up to the nearest integer)
- *Number of trees*: [10,20,...,100]
2. Report the time required to perform cross-validation via `RandomizedSearchCV`. Report the mean and standard deviation of the performance metrics for the best performing model along with its associated hyperparameters. You may use the function `collate_ht_results` for this purpose.
2. Compare the performance of this model to the best RF model constructed in Part 2. Which performs better?

## Part 5: Feature Selection

In [ ]:
Perform a stratified split of the **training data** into feature selection (one-sixth)/training (five-sixths) sets, then:
1. Calculate feature importance on the feature selection subset using *Random Forest Feature Importance (RFFI)*. Produce a bar graph showing each feature's importance and its standard deviation, as determined by RFFI, in descending order.
2. Choosing the **top 6 features**, train NB, SVM, and RF models on this new,smaller training set and test on the validation set.
3. Choosing the **top 11 features**, train NB, SVM, and RF models on this new,smaller training set and test on the validation set.
4. Compare the performance of the models with respect the algorithm used (e.g., NB) and the number of features chosen (6, 11, and all features (those constructed in Part 1). How do they compare?

## Part 6: Final Models and Visualization

In [ ]:
1. Using the full training set (**feature selection + training + validation**), train NB, SVM, and RF models, then apply them to the test set. Your final NB, SVM, and RF models should be named `gnb_final`, `svm_final`, and `rf_final`.
2. Create a bar chart of mean metrics from cross-validation, with standard deviation as an error bar for each model on the same plot. Use the NB model from Part 2, but the optimal models from Parts 3 and 4 for SVM and RF respectively.
3. Plot Receiver Operating Characteristic (ROC) curves for each final model in a single plot.
4. Which of the above metrics best illuminate the difference (if any) in model performance? Why?

## Part 7: Holdout Set Evaluation (Instructor)

In [ ]:
Your final models `gnb_final`, `svm_final`, and `rf_final` will be evaluated on a holdout set by the instructor.